# Load Libraries

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, KFold
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

C:\Users\panko\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\__init__.py:177: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Change the directory

In [2]:
import os
os.chdir("../../../")
# from google.colab import drive
# drive.mount('/content/drive/')
# %cd drive/MyDrive/poleval_emotion/

# Determine constants

In [3]:
TRAIN_DIR = 'data/train/'
IN_FILENAME = 'in_prep_bas.tsv'
EXPECTED_FILENAME = 'expected.tsv'

# Load & preprocess data functions

In [4]:
def load_data(file_path):
    data = pd.read_csv(file_path, sep='\t')
    return data

In [5]:
def preprocess_data(text_series, num_words=5000):
    text_series = text_series.astype(str)
    tokenizer = Tokenizer(num_words=num_words)
    tokenizer.fit_on_texts(text_series)
    sequences = tokenizer.texts_to_sequences(text_series)
    max_len = max(len(x) for x in sequences)
    padded_sequences = pad_sequences(sequences, maxlen=max_len)
    return padded_sequences, tokenizer

# Define the LSTM model

In [6]:
def create_model():
    model = Sequential([
        Embedding(input_dim=5000, output_dim=128),
        LSTM(64),
        Dense(11, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Load & preprocess the data

In [7]:
in_data = load_data(TRAIN_DIR + IN_FILENAME)
expected_data = load_data(TRAIN_DIR + EXPECTED_FILENAME)

In [8]:
X, tokenizer = preprocess_data(in_data['text'])
y = expected_data.values

# Cross-validation setup

In [9]:
num_splits = 5
num_repeats = 5
kf = KFold(n_splits=num_splits, shuffle=True, random_state=1613)

# Main operation

In [10]:
overall_accuracies = []
all_labels_accuracies = []

In [11]:
for repeat in range(num_repeats):
    fold_accuracies = []
    fold_labels_accuracies = []
    fold_no = 1
    for train_index, test_index in kf.split(X):
        print(f"Repeat {repeat+1}, Fold {fold_no} started.")
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = create_model()
        history = model.fit(X_train, y_train, batch_size=32, epochs=16, validation_data=(X_test, y_test), verbose=0)

        # Model evaluation
        scores = model.evaluate(X_test, y_test, verbose=0)
        fold_accuracies.append(scores[1])
        print(f'Score for repeat {repeat+1}, fold {fold_no}: {model.metrics_names[1]} of {scores[1]:.4f}')

        # Calculating the accuracy for each label
        predictions = model.predict(X_test)
        label_accuracies = []
        accuracies_dict = {}
        for i in range(y_test.shape[1]):
            accuracy = np.mean((predictions[:, i] > 0.5) == y_test[:, i])
            label_accuracies.append(accuracy)
            accuracies_dict[expected_data.columns[i]] = [f"{accuracy:.3f}"]

        # Listing the accuracy for each label in table form
        accuracies_df = pd.DataFrame(accuracies_dict, index=['Accuracy'])
        print(accuracies_df)

        # Average accuracy for all labels
        mean_label_accuracy = np.mean(label_accuracies)
        fold_labels_accuracies.append(label_accuracies)
        print(f'Mean accuracy for all 11 labels: {mean_label_accuracy:.4f}\n')

        fold_no += 1

    overall_accuracies.append(fold_accuracies)
    all_labels_accuracies.append(fold_labels_accuracies)
    print()

Repeat 1, Fold 1 started.
Score for repeat 1, fold 1: compile_metrics of 0.1137
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step
            Joy  Trust Anticipation Surprise   Fear Sadness Disgust  Anger  \
Accuracy  0.708  0.788        0.862    0.930  0.950   0.720   0.794  0.825   

         Positive Negative Neutral  
Accuracy    0.710    0.730   0.780  
Mean accuracy for all 11 labels: 0.7996

Repeat 1, Fold 2 started.
Score for repeat 1, fold 2: compile_metrics of 0.1165
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step
            Joy  Trust Anticipation Surprise   Fear Sadness Disgust  Anger  \
Accuracy  0.715  0.791        0.858    0.925  0.950   0.708   0.822  0.830   

         Positive Negative Neutral  
Accuracy    0.724    0.710   0.781  
Mean accuracy for all 11 labels: 0.8013

Repeat 1, Fold 3 started.
Score for repeat 1, fold 3: compile_metrics of 0.1172
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step
            Joy  Trust Anticipation Surprise   Fear Sadness Disgust  Anger  \
Accuracy  0.695  0

# Final results

In [12]:
final_overall_accuracy = np.mean(overall_accuracies)
final_labels_accuracy = np.mean(all_labels_accuracies, axis=(0, 1))
overall_std = np.std(overall_accuracies)
labels_std = np.std(all_labels_accuracies, axis=(0, 1))

In [13]:
print(f'Final average overall accuracy: {final_overall_accuracy:.4f} with std deviation: {overall_std:.4f}')
print("\nFinal average accuracy for each label with their std deviation:")
for label, acc, std in zip(expected_data.columns, final_labels_accuracy, labels_std):
    print(f"{label}: {acc:.4f} (std: {std:.4f})")

Final average overall accuracy: 0.1279 with std deviation: 0.0233

Final average accuracy for each label with their std deviation:
Joy: 0.7043 (std: 0.0095)
Trust: 0.7997 (std: 0.0143)
Anticipation: 0.8571 (std: 0.0093)
Surprise: 0.9232 (std: 0.0070)
Fear: 0.9505 (std: 0.0057)
Sadness: 0.7193 (std: 0.0099)
Disgust: 0.8088 (std: 0.0148)
Anger: 0.8263 (std: 0.0124)
Positive: 0.7120 (std: 0.0095)
Negative: 0.7206 (std: 0.0108)
Neutral: 0.7858 (std: 0.0117)
